# <b><span style='color:#F1A424'>|</span> GOOGLE: <span style='color:#F1A424'>GEMMA</span><span style='color:#ABABAB'> </span></b> 

***

This notebook presents an accessible method for fine-tuning the Google Gemma 2b model to address fundamental inquiries regarding the Python Language. It initiates by configuring the environment with 'jax' as the backend and TPUs for optimized performance. Three distinct prompts were formulated to evaluate the model's output before and after fine-tuning, facilitating a comprehensive comparison of results. Diverse datasets were sourced and imported to refine the model effectively. Following this, the model underwent training with predefined parameters, culminating in inference analysis. Remarkably, even with a modest amount of data utilized for fine-tuning, a significant enhancement in model performance was observed. Consequently, the Gemma model emerges as a versatile tool adaptable to a myriad of tasks through straightforward methodologies.

The main steps of fine-tuning in this notebook can be summed up as follows:
- Setting up Environment of VM v3-8 TPUs with 'jax' backend
- Importing Gemma 2b-model
- Setting up standard question templates for comparison
- Getting the Required Data
- Fine-tuning Gemma with Low Rank Adaption(LoRa)
- Inference after Fine-tuning results

### <b><span style='color:#F1A424'>Table of Contents</span></b> <a class='anchor' id='top'></a>
<div style=" background-color:#3b3745; padding: 13px 13px; border-radius: 8px; color: white">
<li><a href="#Summary">Summary</a></li>
<li><a href="#Installing_Libraries">Installing Libraries</a></li>
<li><a href="#Setting_Environment">Setting Environment</a></li>
<li><a href="#TPU_Distribution">TPU Distribution</a></li>
<li><a href="#Importing_Gemma">Importing Gemma</a></li>
<li><a href="#Templates">Template</a></li>
<li><a href="#Initial_Response">Initial Response of Gemma</a></li>
<li><a href="#Getting_Data_for_Fine_Tuning">Getting Data for Fine-Tuning</a></li>
<li><a href="#Fine_Tuning">Fine Tuning</a></li>
<li><a href="#Inference_After_Fine_Tuning">Inference After Fine Tuning</a></li>
<li><a href="#Conclusion">Conclusion</a></li>
<li><a href="#References">References</a></li>
</div>



**To submit to this competition fill out the Google Form [here](https://www.kaggle.com/kaggle-assistant-submissions).**




# <b><span style='color:#F1A424'>|</span>Installing Libraries</b><a class='anchor' id='Installing_Libraries'></a> [↑](#top) 

***



In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q tensorflow-cpu
!pip install -q -U keras-nlp tensorflow-hub
!pip install -q -U keras>=3
!pip install -U tensorflow-text

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.8.1 requires keras-core, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.3.2 which is incompatible.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.16.2 which is incompatible.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.16.1 requires keras>=3.0.0, but you have keras 2.15.0 which is incompatible.
tensorflow-cpu 2.16.1 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tensorflow-cpu 2.16.1 requires tensorboard<2.17,>=2.16, but you have tensorboard 2.15.2 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.
tensorflow-cpu 2.16.1 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tensorflow-cpu 2.16.1 requires tensorboard<2.17,>=2.16, but you have tensorboard 2.15.2 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.2 MB ? eta -:--:--

...

      Successfully uninstalled tensorflow-2.15.0


  Attempting uninstall: tensorflow-text
    Found existing installation: tensorflow-text 2.15.0


    Uninstalling tensorflow-text-2.15.0:


      Successfully uninstalled tensorflow-text-2.15.0



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import jax

jax.devices()

E0414 18:49:29.566315004     123 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2024-04-14T18:49:29.566298914+00:00", grpc_status:2}


[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

# <b><span style='color:#F1A424'>|</span> Setting Environment</b><a class='anchor' id='Setting_Environment'></a> [↑](#top) 

***


In [3]:
import os
# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 90% of TPU memory to minimize memory fragmentation and allocation
# overhead
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

# <b><span style='color:#F1A424'>|</span> TPU Distribution</b><a class='anchor' id='TPU_Distribution'></a> [↑](#top) 

***

To start using the model with the weights and data spread out over TPUs, you begin by making a new DeviceMesh. This DeviceMesh is a group of devices, like computers, set up to work together for big tasks. It was brought into Keras 3 as part of the way they handle sharing work across many machines.

This way of sharing work lets us split up the data and model, which means we can make big machine learning models work faster by spreading the work across lots of machines. It uses the main framework (like JAX) to split up the program and data following specific rules, a bit like dividing up a big job into smaller pieces.

In [4]:
import keras
import keras_nlp
import tqdm

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Create a device mesh with (1, 8) shape so that the weights are sharded across
# all 8 TPUs.
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices())

In the distribution API, LayoutMap tells us how to divide or copy the weights and data across different machines. It uses string keys, like "token_embedding/embeddings" mentioned below. These keys work like patterns to find specific parts of the data. When it finds a match, it splits that data across 8 TPUs (Tensor Processing Units), which are special types of hardware. Other parts of the data that don't match these patterns are just copied completely to each machine.

In [6]:
model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# Weights that match 'token_embedding/embeddings' will be sharded on 8 TPUs
layout_map["token_embedding/embeddings"] = (None, model_dim)
# Regex to match against the query, key and value matrices in the decoder
# attention layers
layout_map["decoder_block.*attention.*(query|key|value).*kernel"] = (None, model_dim, None)

layout_map["decoder_block.*attention_output.*kernel"] = (None, None, model_dim)
layout_map["decoder_block.*ffw_gating.*kernel"] = (model_dim, None)
layout_map["decoder_block.*ffw_linear.*kernel"] = (None, model_dim)

# <b><span style='color:#F1A424'>|</span> Importing Gemma</b><a class='anchor' id='Importing_Gemma'></a> [↑](#top) 

***

With ModelParallel, you can split the weights or data of the model across all devices connected to the DeviceMesh. For example, with the Gemma 2B model, some of its weights are divided across 8 TPU chips based on the layout_map we set earlier. Now, let's load the model so it's spread out across all these devices.

In [7]:
model_parallel = keras.distribution.ModelParallel(device_mesh, layout_map, batch_dim_name="batch")

keras.distribution.set_distribution(model_parallel)
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma/keras/gemma_2b_en/2")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


### <b><span style='color:#F1A424'>Partition Check</span></b>

In [8]:
decoder_block_1 = gemma_lm.backbone.get_layer('decoder_block_1')
print(type(decoder_block_1))
for variable in decoder_block_1.weights:
    print(f'{variable.path:<58}  {str(variable.shape):<16}  {str(variable.value.sharding.spec)}')

<class 'keras_nlp.src.models.gemma.gemma_decoder_block.GemmaDecoderBlock'>
decoder_block_1/pre_attention_norm/scale                    (2048,)           PartitionSpec(None,)
decoder_block_1/attention/query/kernel                      (8, 2048, 256)    PartitionSpec(None, 'model', None)
decoder_block_1/attention/key/kernel                        (1, 2048, 256)    PartitionSpec(None, 'model', None)
decoder_block_1/attention/value/kernel                      (1, 2048, 256)    PartitionSpec(None, 'model', None)
decoder_block_1/attention/attention_output/kernel           (8, 256, 2048)    PartitionSpec(None, None, 'model')
decoder_block_1/pre_ffw_norm/scale                          (2048,)           PartitionSpec(None,)
decoder_block_1/ffw_gating/kernel                           (2048, 16384)     PartitionSpec('model', None)
decoder_block_1/ffw_gating_2/kernel                         (2048, 16384)     PartitionSpec('model', None)
decoder_block_1/ffw_linear/kernel                           (

# <b><span style='color:#F1A424'>|</span> Template</b><a class='anchor' id='Template'></a> [↑](#top) 

***

We're going to create a template for testing both before and after fine-tuning. This will let us compare the results. We'll ask questions to find out what's different between the outputs generated before and after fine-tuning.

* What are the key features of Python?
* How is python different from other programming languages like R, C, C++?
* Is Python the most robust language in developing machine learning algorithms?

In [9]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

In [10]:
prompt_1 = template.format(
    instruction="What are the key features of Python?",
    response="",
)

prompt_2 = template.format(
    instruction="How is python different from other programming languages like R, C, C++?",
    response="",
)

prompt_3 = template.format(
    instruction="Is Python the most robust language in developing machine learning algorithms?",
    response="",
)

# <b><span style='color:#F1A424'>|</span> Initial Response of Gemma</b><a class='anchor' id='Initial_Response'></a> [↑](#top) 

***

The result of the model on these selected seems to be a mix of good and bad results. Forexample, in the first question, it quickly runs out of vocaubalry and keeps on repeating the same response again and again whereas, in the latter questions, it does seem to be giving fine results. Let's see if we can improve it.

In [11]:
%%time 

print(gemma_lm.generate(prompt_1, max_length=250))

Instruction:
What are the key features of Python?

Response:
Python is a high-level, general-purpose, interpreted, object-oriented programming language. It is a popular language for web development, data analysis, and scientific computing. Python is easy to learn and use, and has a large and active community of developers. It is also widely used in industry, academia, and government.

What are the benefits of using Python?

Response:
Python is a powerful and versatile programming language that is easy to learn and use. It has a large and active community of developers, and is widely used in industry, academia, and government. Python is also a popular choice for web development, data analysis, and scientific computing.

What are the features of Python?

Response:
Python is a high-level, interpreted, object-oriented programming language. It is a popular language for web development, data analysis, and scientific computing. Python has a large and active community of developers, and is wid

In [12]:
print(gemma_lm.generate(prompt_2, max_length=250))

Instruction:
How is python different from other programming languages like R, C, C++?

Response:
Python is a high-level, general-purpose programming language. It is interpreted, object-oriented, and high-level. It is a general-purpose programming language that is easy to learn and use. It is a high-level, general-purpose programming language. It is interpreted, object-oriented, and high-level. It is a general-purpose programming language that is easy to learn and use. It is a high-level, general-purpose programming language. It is interpreted, object-oriented, and high-level. It is a general-purpose programming language that is easy to learn and use. It is a high-level, general-purpose programming language. It is interpreted, object-oriented, and high-level. It is a general-purpose programming language that is easy to learn and use. It is a high-level, general-purpose programming language. It is interpreted, object-oriented, and high-level. It is a general-purpose programming language 

In [13]:
print(gemma_lm.generate(prompt_3, max_length=250))

Instruction:
Is Python the most robust language in developing machine learning algorithms?

Response:
Yes, Python is the most robust language in developing machine learning algorithms.

Explanation:
Python is a general-purpose programming language that is used for a wide range of applications, including web development, data analysis, machine learning, and scientific computing. It is known for its readability, simplicity, and versatility, making it a popular choice for developers and data scientists.

Python has a large and active community of developers and libraries, which makes it easy to find and use tools and frameworks for machine learning. It also has a wide range of libraries and frameworks for machine learning, including scikit-learn, TensorFlow, and PyTorch, which make it easy to build and train machine learning models.

Overall, Python is a powerful and versatile language that is well-suited for developing machine learning algorithms. It is widely used in the field and has a

# <b><span style='color:#F1A424'>|</span> Getting Data for Fine-Tuning</b><a class='anchor' id='Getting_Data_for_Fine_Tuning'></a> [↑](#top) 

***

The result above that we asked kaggle shows some mix performance. We can ,however, improve the results by simply adding some additional data to increase the overall vocabulary. 

The first 2 datasets are composed of python questions and answers from Reddit and Stackoverflow. This [notebook](https://www.kaggle.com/code/mohib94/gemma-data-scrap) was used to scrap the data from the platforms. The 3rd dataset has been taken from [here](https://www.kaggle.com/datasets/chinmayadatt/dataset-python-question-answer).


In [14]:
import pandas as pd
reddit = pd.read_csv('/kaggle/input/gemma-data-scrap/reddit_questions_answers.csv')
stack_overflow = pd.read_csv('/kaggle/input/gemma-data-scrap/stack_questions_answers.csv')
stack_overflow.drop('Question ID', axis =1 , inplace = True)
dataset = pd.read_csv('/kaggle/input/python-question-answers/Dataset_Python_Question_Answer.csv')

data_list = [reddit , stack_overflow, dataset]
train = pd.concat(data_list)
train.head()

,Question,Answer
0,Sunday Daily Thread: What's everyone working o...,I'm writing a library for hand crafted SQL que...
1,Saturday Daily Thread: Resource Request and Sh...,This is a Fakespot Reviews Analysis bot. Fakes...
2,Designing a Pure Python Web Framework,I've been using NiceGUI for a year and it's be...
3,I made a free easy-to-use toast notification l...,"Cool, seems good"
4,Looking for a Object detection Library or Api ...,What about the Segment Anything model?


In [15]:
#join dataset columns
combined_train=pd.DataFrame(columns=["QA"])
combined_train=train["Question"]+train["Answer"]

# <b><span style='color:#F1A424'>|</span> Fine Tuning</b><a class='anchor' id='Fine_Tuning'></a> [↑](#top) 

***


When fine-tuning a model, we typically adjust its parameters to suit new tasks. While fine-tuning large models we can use an effective method called Low Rank Adaptation(LoRa) to adjust the overall parameters to scale the training. However, since we are already using the smaller version of Gemma, there is no need to use it but rather only limit the input sequence length to mantain in on scale.

In [16]:
%%time
# Limit the input sequence length to 64 to control memory usage.
gemma_lm.preprocessor.sequence_length = 64

# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.summary()
gemma_lm.fit(combined_train, epochs=3)

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

...

16/18 ━━━━━━━━━━━━━━━━━━━━ 1s 517ms/step - loss: 0.8415 - sparse_categorical_accuracy: 0.7552

17/18 ━━━━━━━━━━━━━━━━━━━━ 0s 517ms/step - loss: 0.8447 - sparse_categorical_accuracy: 0.7556

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - loss: 0.8474 - sparse_categorical_accuracy: 0.7560

18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 499ms/step - loss: 0.8499 - sparse_categorical_accuracy: 0.7563


CPU times: user 2min 44s, sys: 7.35 s, total: 2min 51s
Wall time: 1min 35s


# <b><span style='color:#F1A424'>|</span> Inference After Fine Tuning</b><a class='anchor' id='Inference_After_Fine_Tuning'></a> [↑](#top) 

***


In [17]:
%%time
print(gemma_lm.generate(prompt_1, max_length=250))

Instruction:
What are the key features of Python?

Response:
Python is an extremely malleable language. It is very good at string manipulation and regular expressions. It is also very good at dynamic typing. Additionally, it has excellent tooling support.
CPU times: user 35.4 s, sys: 416 ms, total: 35.8 s
Wall time: 17.1 s


In [18]:
%%time
print(gemma_lm.generate(prompt_2, max_length=250))

Instruction:
How is python different from other programming languages like R, C, C++?

Response:
Python is a very different language from other programming languages like R, C, C++. Python is dynamically typed, meaning the compiler doesn't know the exact types of variables or functions at compile time. This allows python to be much more permissive with typing and annotation. Additionally, python is interpreted, meaning the code is executed by the python interpreter as it is read. This allows python to be much more permissive with strictness of typing and annotation. Finally, python is much more permissive with module isolation and packaging. With venv you can create isolated environments for each project, with pip you can install different versions of python on the same machine. Additionally, venv allows you to define different requirements for different projects, so you can have python package foo
CPU times: user 30.3 ms, sys: 15.6 ms, total: 45.9 ms
Wall time: 789 ms


In [19]:
%%time
print(gemma_lm.generate(prompt_3, max_length=250))

Instruction:
Is Python the most robust language in developing machine learning algorithms?

Response:
No doubt Python is very good at it. But I would want to see some benchmarks before I could make a firm opinion.
CPU times: user 35.8 ms, sys: 6.61 ms, total: 42.4 ms
Wall time: 177 ms


# <b><span style='color:#F1A424'>|</span> Conclusion</b><a class='anchor' id='Conclusion'></a> [↑](#top) 

***

Therefore, it's evident that through a relatively straightforward method, we can enhance the performance of the model, particularly concerning Python language queries. It is important to mention that keras implementation of the Gemma model is memory inefficient requiring us to use the 2b with limited sequence length. If we switch to Pytorch or Transformer, we can radically improve the model results alongwith the incorporation of LoRa. In the initial question, we observe significant improvement compared to mere repetition seen previously. Additionally, there's a noticeable enhancement in the structure of the other two questions. This progress was achieved through fine-tuning with selected questions and answers sourced from Reddit, StackOverflow, and open-source platforms. Thus, the remarkable ease with which the model can be trained results in a substantial improvement in overall performance making Gemma an ideal choice for wide usage. 

# <b><span style='color:#F1A424'>|</span> References</b><a class='anchor' id='References'></a> [↑](#top) 

***

https://ai.google.dev/gemma/docs/distributed_tuning